<a href="https://colab.research.google.com/github/scottwmwork/DS-Unit-2-Applied-Modeling/blob/master/module4/assignment_applied_modeling_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Applied Modeling, Module 4

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] Continue to iterate on your project: data cleaning, exploratory visualization, feature engineering, modeling.
- [ ] Make a Shapley force plot to explain at least 1 individual prediction.
- [ ] Share at least 1 visualization on Slack.

(If you haven't completed an initial model yet for your portfolio project, then do today's assignment using your Tanzania Waterpumps model.)

## Stretch Goals
- [ ] Make Shapley force plots to explain at least 4 individual predictions.
    - If your project is Binary Classification, you can do a True Positive, True Negative, False Positive, False Negative.
    - If your project is Regression, you can do a high prediction with low error, a low prediction with low error, a high prediction with high error, and a low prediction with high error.
- [ ] Use Shapley values to display verbal explanations of individual predictions.
- [ ] Use the SHAP library for other visualization types.

The [SHAP repo](https://github.com/slundberg/shap) has examples for many visualization types, including:

- Force Plot, individual predictions
- Force Plot, multiple predictions
- Dependence Plot
- Summary Plot
- Summary Plot, Bar
- Interaction Values
- Decision Plots

We just did the first type during the lesson. The [Kaggle microcourse](https://www.kaggle.com/dansbecker/advanced-uses-of-shap-values) shows two more. Experiment and see what you can learn!


## Links
- [Kaggle / Dan Becker: Machine Learning Explainability — SHAP Values](https://www.kaggle.com/learn/machine-learning-explainability)
- [Christoph Molnar: Interpretable Machine Learning — Shapley Values](https://christophm.github.io/interpretable-ml-book/shapley.html)
- [SHAP repo](https://github.com/slundberg/shap) & [docs](https://shap.readthedocs.io/en/latest/)

In [30]:
import sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install packages in Colab
    !pip install -r requirements.txt
    !pip install category_encoders==2.0.0
    !pip install pandas-profiling==2.3.0
    !pip install plotly==4.1.1
    !pip install shap

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [0]:
#importing the dataframe
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/scottwmwork/datasets/master/tmdb_5000_movies.csv')

In [0]:
#Add columns for year month and day via pd.to_datetime()
df['release_date'] = pd.to_datetime(df['release_date'],infer_datetime_format = True)
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month
df['release_day'] = df['release_date'].dt.month
df = df.drop(columns = 'release_date')

In [0]:
#Isolate the test set
test = df[df['release_year'] == 2016]
y_test = test['revenue']
X_test = test.drop(columns = 'revenue')


#Exclude test set from data
dfn = df[df['release_year'] != 2016]

In [0]:
#Create train and validation data 
from sklearn.model_selection import train_test_split
train, val = train_test_split(dfn, train_size = .80, test_size = 0.20, random_state = 42)

y_train = train.revenue
X_train = train.drop(columns = 'revenue')

y_val = val.revenue
X_val = val.drop(columns = 'revenue')

import ast
import numpy as np

#Wrangle function to engineer, drop, and add columns
def wrangle(X):
  
  X = X.copy()
  X = X.reset_index()

  #Make genres column usable
  genre = []
  for x in X['genres']:
    if x == '[]':
      genre.append(np.nan)
    else:
      temp = ast.literal_eval(x) 
      genre.append(temp[0]['name']) #grabs first genre in list of dictionaries
    

  #Features to not include:
  X = X.drop(columns = ['genres','homepage','keywords','overview','production_companies','production_countries','spoken_languages','tagline'])
  
  #Engineer features:
  
  #original title is same as title?    
  title_changed = []
  for x in range(0,len(X['title'])):
    if X['title'][x] == X['original_title'][x]:
       title_changed.append(0)
    else:
       title_changed.append(1)
  
  #length of title
  length_of_title = []
  for x in X['title']:
    length_of_title.append(len(x))
  
  #Add features to dataframe
  X['title_changed'] = title_changed
  X['length_of_title'] = length_of_title
  X['genre_first_listed'] = genre
   
  
  return X

In [0]:
#Wrangle data
X_test = wrangle(X_test)
X_val = wrangle(X_val)
X_train = wrangle(X_train)

In [53]:
from sklearn.impute import SimpleImputer 
from sklearn.linear_model import LinearRegression
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder

X_train_encoded = ce.OneHotEncoder(use_cat_names = True).fit_transform(X_train)
X_train_encoded = SimpleImputer().fit_transform(X_train_encoded)
model = LinearRegression()
model.fit(X_train_encoded,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [56]:
X_train.columns

Index(['index', 'budget', 'id', 'original_language', 'original_title',
       'popularity', 'runtime', 'status', 'title', 'vote_average',
       'vote_count', 'release_year', 'release_month', 'release_day',
       'title_changed', 'length_of_title', 'genre_first_listed'],
      dtype='object')

In [63]:
def predict(index, budget, id, original_language, original_title,
       popularity, runtime, status, title, vote_average,
       vote_count, release_year, release_month, release_day,
       title_changed, length_of_title, genre_first_listed):
    df = pd.DataFrame(
        data=[[index, budget, id, original_language, original_title,
       popularity, runtime, status, title, vote_average,
       vote_count, release_year, release_month, release_day,
       title_changed, length_of_title, genre_first_listed]], 
        
        columns=['index', 'budget', 'id', 'original_language', 'original_title',
       'popularity', 'runtime', 'status', 'title', 'vote_average',
       'vote_count', 'release_year', 'release_month', 'release_day',
       'title_changed', 'length_of_title', 'genre_first_listed']
    )
    pred = model.predict(df)[0]
    
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(df)
    
    feature_names = df.columns
    feature_values = df.values[0]
    shaps = pd.Series(shap_values[0], zip(feature_names, feature_values))
    
    shap.initjs()
    return shap.force_plot(
        base_value=explainer.expected_value,
        shap_values=shap_values, 
        features=df
    )

predict(index = 10, budget = 1000000, id = 1234, original_language = 1.2, original_title = 100,
       popularity = 123124, runtime = 2.4, status = 100, title = 100, vote_average = 12341,
       vote_count = 21324, release_year = 2019, release_month = 3, release_day = 9,
       title_changed = 20, length_of_title = 9, genre_first_listed = 100)

ValueError: ignored